In [16]:
#importando

import requests

from bs4 import BeautifulSoup

import pandas as pd

import plotly.graph_objects as Dash

In [17]:
!pip install beautifulsoup4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [18]:
#carregar paginas

#url para buscar os dados

site = 'http://www.ssp.sp.gov.br/Estatistica/ViolenciaMulher.aspx'

Pagina = requests.get( site )

In [19]:
Pagina

<Response [200]>

In [20]:
#essa reposta acima, Response 200, signifca que deu tudo certo.

In [21]:
#verificando o tipo

type(Pagina)

requests.models.Response

In [22]:
Pagina.text

'\r\n\r\n<!DOCTYPE html>\r\n\r\n<html xmlns="http://www.w3.org/1999/xhtml" ng-app="modPortal">\r\n<head ng-controller="ctrlHead"><meta name="fragment" content="!" /><meta http-equiv="Content-Type" content="text/html; charset=UTF-8" /><meta http-equiv="X-UA-Compatible" content="IE=edge" /><meta name="viewport" content="width=device-width, initial-scale=1" />\r\n\t<!-- The above 3 meta tags *must* come first in the head; any other head content must come *after* these tags -->\r\n\t<title>\r\n\tSSP\r\n</title>\r\n\r\n\t<!-- Bootstrap core CSS -->\r\n\t<link href="/Content/bootstrap.min.css" rel="stylesheet" />\r\n\t<!-- Bootstrap theme -->\r\n\t<link href="/Content/bootstrap-theme.min.css" rel="stylesheet" />\r\n\t<!-- IE10 viewport hack for Surface/desktop Windows 8 bug -->\r\n\t<link href="/Content/ie10-viewport-bug-workaround.css" rel="stylesheet" />\r\n\t<!-- Custom styles for this template -->\r\n\t<link href="/Content/theme.css" rel="stylesheet" /><link href="/Content/SearchForm.css

In [23]:
#bs4 vai interpretar o arquivo html
Coleta = BeautifulSoup( Pagina.text, 'html.parser')

In [24]:
type(Coleta)

bs4.BeautifulSoup

In [25]:
Coleta


<!DOCTYPE html>

<html ng-app="modPortal" xmlns="http://www.w3.org/1999/xhtml">
<head ng-controller="ctrlHead"><meta content="!" name="fragment"/><meta content="text/html; charset=utf-8" http-equiv="Content-Type"/><meta content="IE=edge" http-equiv="X-UA-Compatible"/><meta content="width=device-width, initial-scale=1" name="viewport"/>
<!-- The above 3 meta tags *must* come first in the head; any other head content must come *after* these tags -->
<title>
	SSP
</title>
<!-- Bootstrap core CSS -->
<link href="/Content/bootstrap.min.css" rel="stylesheet"/>
<!-- Bootstrap theme -->
<link href="/Content/bootstrap-theme.min.css" rel="stylesheet"/>
<!-- IE10 viewport hack for Surface/desktop Windows 8 bug -->
<link href="/Content/ie10-viewport-bug-workaround.css" rel="stylesheet"/>
<!-- Custom styles for this template -->
<link href="/Content/theme.css" rel="stylesheet"/><link href="/Content/SearchForm.css" rel="stylesheet"/><link href="/Content/proxima.css" rel="stylesheet"/><link href="/C

In [26]:
#coletando o titulo da pagina
Coleta.title

<title>
	SSP
</title>

In [31]:
#vai localizar todas as tabelas dentro da página mas apenas a da referidas classe indicada em attrs
Tabelas = Coleta.find_all('table', attrs={'class' : 'table table-striped table-hover table-condensed'} )

print(f'Foi localizado {len(Tabelas)} Tabelas')

Foi localizado 130 Tabelas


In [38]:
Tabelas[0].find_all('tr')[2].find_all('td')[4].find_all('span')[0].text

'11'

In [56]:
#funçoes para datas
import datetime
from dateutil.relativedelta import relativedelta


#variaveis
#foram criadas essas variáveis para receberos valores
Rotulos = []
Dados = []
Periodo = []

#registro inicial do site
Registro_Inicial = datetime.datetime( 2022, 6, 1)


#looping nas tabelas
for posicao,consulta in enumerate(Tabelas):

  #Linha da tabela
  Linha = consulta.find_all('tr')[2]

  #Celula
  Rotulo = Linha.find_all('td')[4]

  #nomenclatura do feminicidio
  Label = Linha.find_all('td')[0].text

  #informacao (qtde de assassinatos)
  Informacao = Rotulo.find_all('span')[0].text

  #calculando a data
  Data = Registro_Inicial - relativedelta( months=posicao)
  if Label == 'FEMINICÍDIO':
    #salvando a informação
    Dados.append( Informacao )
    Rotulos.append( Label )
    Periodo.append( Data )
  else:
    pass

  #print(posicao, Informacao)
Dicionario = {
    'Indicador': Rotulos,
    'Valores' : Dados,
    'Período' : Periodo
}

Tabela_Web = pd.DataFrame(Dicionario)
Tabela_Web.head(10)

,Indicador,Valores,Período
0,FEMINICÍDIO,11,2022-06-01
1,FEMINICÍDIO,12,2022-05-01
2,FEMINICÍDIO,10,2022-04-01
3,FEMINICÍDIO,19,2022-03-01
4,FEMINICÍDIO,11,2022-02-01
5,FEMINICÍDIO,12,2022-01-01
6,FEMINICÍDIO,10,2021-12-01
7,FEMINICÍDIO,9,2021-11-01
8,FEMINICÍDIO,9,2021-10-01
9,FEMINICÍDIO,6,2021-09-01


In [58]:
#verificando os últimos registros
Tabela_Web.tail()

,Indicador,Valores,Período
49,FEMINICÍDIO,16,2018-05-01
50,FEMINICÍDIO,14,2018-04-01
51,FEMINICÍDIO,8,2018-03-01
52,FEMINICÍDIO,8,2018-02-01
53,FEMINICÍDIO,5,2018-01-01


In [59]:
#fazer as analises de estatisticas
Tabela_Web.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54 entries, 0 to 53
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Indicador  54 non-null     object        
 1   Valores    54 non-null     object        
 2   Período    54 non-null     datetime64[ns]
dtypes: datetime64[ns](1), object(2)
memory usage: 1.4+ KB


In [60]:
#o programa nao reconheceu a coluna valores como numero e sim, como texto, fazer modificação

In [61]:
#convertendo texto em número com o numeric
Tabela_Web['Valores'] = pd.to_numeric(Tabela_Web['Valores'])

In [62]:
Tabela_Web.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54 entries, 0 to 53
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Indicador  54 non-null     object        
 1   Valores    54 non-null     int64         
 2   Período    54 non-null     datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 1.4+ KB


In [66]:
#setando o index, ou seja, transformando a a coluna em indica
Tabela_Web.set_index('Período', inplace=True)


KeyError: ignored

In [67]:
Tabela_Web.head()

,Indicador,Valores
Período,,
2022-06-01,FEMINICÍDIO,11
2022-05-01,FEMINICÍDIO,12
2022-04-01,FEMINICÍDIO,10
2022-03-01,FEMINICÍDIO,19
2022-02-01,FEMINICÍDIO,11


In [76]:
#grafico
Figura = Dash.Figure()

Figura.add_trace(
    Dash.Scatter(
        x = Tabela_Web.index,
        y = Tabela_Web.Valores,
        mode = 'lines', #tipo de gráfico
        name='Quantidade',
        marker_color='#FF7F0E'
    )
)
Figura.update_layout(
    title='Historico de Feminicidio para o Estado de São Paulo',
    titlefont_size=20
)

Figura.show()